In [1]:
file_path="F:/eegonlinedata/22_INSIGHT2_170315_2022.11.09T12.53.37+05.30.edf"
event_path="F:/eegonlinedata/rahul_sai_results.csv"

In [2]:
from mne_realtime import LSLClient, MockLSLStream
import pylsl as lsl
import matplotlib.pyplot as plt
import time
from scipy.signal import wiener
from scipy.signal import welch
import numpy as np
import pyautogui
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
# !nvidia-smi
import torch
import mne



In [3]:
#find host id
streams = lsl.resolve_streams(wait_time=min(0.1, 3))

for stream_info in streams:
            print(stream_info.source_id())
###########

host= 'INSIGHT2-A3D20043'
wait_max=5
client = LSLClient( info=None,host=host, wait_max=wait_max)
client.start()

   


INSIGHT2-A3D20043
Client: Waiting for server to start
Looking for LSL stream INSIGHT2-A3D20043...
Found stream 'EmotivDataStream-EEG' via INSIGHT2-A3D20043...
Client: Connected


In [4]:
#RECORDED EEG PROCESSING

# df = pd.read_csv(event_path)

npe= np.genfromtxt(event_path, delimiter=',')
npevents=np.zeros((npe.shape[0],3))
npevents[:,0]=npe[:,0]*128
npevents[:,2]=npe[:,1]
npevents=npevents.astype(int)

datax=mne.io.read_raw_edf(file_path,preload=True)
datax.annotations

#channel location
channelnames=datax.ch_names
listo=list(set(channelnames)-set(['AF3', 'T7', 'Pz', 'T8', 'AF4']))
# datax.info['bads'] += listo
datax.drop_channels(listo)
#filter
datax.filter(l_freq=1,h_freq=45)
#time remove
datax.crop(tmin=5)
#rereferencing data\
datax.set_eeg_reference()
event_dict={
 'rh':12,
 'lh':13,
 'rl':14,
 'll':11,
}


Extracting EDF parameters from F:\eegonlinedata\22_INSIGHT2_170315_2022.11.09T12.53.37+05.30.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 209663  =      0.000 ...  1637.992 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 423 samples (3.305 sec)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.0s finished


In [5]:
from mne.decoding import CSP # Common Spatial Pattern Filtering
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import sklearn.model_selection as model_selection


tmin, tmax = -.5, 2
# left_hand = 769,right_hand = 770,foot = 771,tongue = 772
event_id = dict({ 'rh':12, 'lh':13, 'rl':14, 'll':11,})
epochs = mne.Epochs(datax, npevents, event_id, tmin, tmax, proj=True, baseline=None, preload=True)
# epochs=mne.make_fixed_length_epochs(datax,duration=25,overlap=0)
# epochs=epochs.get_data()


labels = epochs.events[:,-1] - 11  
enc = OneHotEncoder()
Yall = enc.fit_transform(labels.reshape(-1,1)).toarray()

Y11= (epochs.events[:,-1]==11)*1
Y11= enc.fit_transform(Y11.reshape(-1,1)).toarray()


data = epochs.get_data()
print("data",data.shape)
# XallF = np.concatenate((std(data), ptp(data), var(data), minim(data), maxim(data), argminim(data), argmaxim(data), mean_square(data), rms(data),abs_diffs_signal(data), skewness(data),  kurtosis(data),concatenate_features(data)
# ), axis=1)
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler() 
# Xscaled = scaler.fit_transform(XallF)
chnls1=1
data1=data[:,chnls1,0:128]
    
# data2=data1.reshape((:,128))


X_train2, X_test2, y_train2, y_test2 = model_selection.train_test_split(data1, Y11, train_size=0.80, test_size=0.20, random_state=101)
# X_train=X_train.reshape(224,641*5)
# X_test=X_test.reshape(56,641*5)

X_train, X_test, y_train, y_test  =  X_train2, X_test2, y_train2, y_test2
# X_val, y_val = X_test,y_test

# print(Xall.shape,Yall.shape, Y11.shape,type(Y11), type(Xall), type(Yall))
print( X_train2.shape, y_test2.shape)
print(set(labels))

trX = torch.from_numpy(X_train).float()
teX = torch.from_numpy(X_test).float()
trY = torch.from_numpy(y_train).long()
teY = torch.from_numpy(y_test).long()
trY = torch.from_numpy(y_train).float()
teY = torch.from_numpy(y_test).float()

print(trX.shape,trX.dtype,trY.shape,trY.dtype,labels.shape)


Not setting metadata
280 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 280 events and 321 original time points ...
0 bad epochs dropped
data (280, 5, 321)
(224, 128) (56, 2)
{0, 1, 2, 3}
torch.Size([224, 128]) torch.float32 torch.Size([224, 2]) torch.float32 (280,)


In [6]:
# Make Neural Network
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.i2h = nn.Linear(input_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)
        
        self.logsoftmax = nn.LogSoftmax(dim=1)
        self.sigm = nn.Sigmoid()

        
    def forward(self, x):
        x = self.sigm(self.i2h(x))
        x = self.sigm(self.h2h(x))
        x = self.sigm(self.h2o(x))
        return x

    
net = Net(128, 20, 2)
epochsN = 10
batch_size = 1
learning_rate = 0.1
momentum = .01


#define loss and optimizer
criterion=nn.BCELoss()
# optimizer=optim.RMSprop(net.parameters(),lr=1e-4),

# criterion = nn.NLLLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)



#training function

def train(model, criterion, optimizer, x, y):
    x = Variable(x, requires_grad=False)
    y = Variable(y, requires_grad=False)
    # print(x.shape,y.shape,y)
    # Reset gradient
    optimizer.zero_grad()

    # Forward
    fx = model.forward(x)
    loss = criterion(fx, y)
    # print(loss,fx,y)
    
    # Backward
    loss.backward()

    # Update parameters
    optimizer.step()

    return loss.data


#make predictions

def predict(model, x, y):
    x = Variable(x, requires_grad=False)
    outputs = model(x)
    _, predicted = torch.max(outputs.data, 1) #for each output, get the predicted value (torch.max returns (index, value) tuple)
    _,in_real= torch.max(y,1)
    correct = (predicted == in_real) #how many predicted values equal the labels
    return correct.sum() 



In [7]:
#run the training

num_examples = trX.shape[0]
# num_batches = num_examples // batch_size
num_batches=1

for e in range(1, epochsN+1):
    loss = 0.
    for k in range(num_batches):
        start, end = k * batch_size, (k + 1) * batch_size
        loss += train(net, criterion, optimizer, trX[start:end], trY[start:end])
    correct = predict(net, teX, teY)
#     plot_loss.append(loss/num_batches)
#     plot_correct.append(correct/teX.shape[0]*100)
#     print("Epoch %02d, loss = %f, accuracy = %.2f%%" % (e, loss / num_batches, correct/teX.shape[0]*100))

In [8]:
sfreq=128
epoch = client.get_data_as_epoch(n_samples=sfreq,picks=[3,4,5,6,7]);
data = epoch.get_data();
#     print(data.shape, np.mean(data))
data = wiener(data, (1,1, 40));
chnls1 = [1]
data1 = data[:,chnls1,:]
data2 = data1.reshape((1,128))
data3 = torch.from_numpy(data2).float()
output = net(data3)

print(output)

Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
tensor([[0.6925, 0.2649]], grad_fn=<SigmoidBackward0>)


In [9]:
# Initiate EyeTracking Pipeline

"""
Stream Pupil gaze coordinate data using zmq to control a mouse with your eye.
Please note that marker tracking must be enabled, and in this example we have named the surface "screen."
You can name the surface what you like in Pupil capture and then write the name of the surface you'd like to use on line 17.
"""
print("start")
# specify the name of the surface you want to use
surface_name = "Surface1"

## install dependencies
# pip3 install zmq msgpack pyuserinput

import zmq
from msgpack import loads
import subprocess as sp
from platform import system

try:
    from pymouse import PyMouse
except ImportError:
    msg = """
    Please install PyMouse from PyUserInput
    https://github.com/PyUserInput/PyUserInput

    pip install PyUserInput
    """
    print(msg)
    exit(1)


m = PyMouse()


context = zmq.Context()
# open a req port to talk to pupil
addr = "127.0.0.1"  # remote ip or localhost
req_port = "50020"  # same as in the pupil remote gui
req = context.socket(zmq.REQ)
req.connect("tcp://{}:{}".format(addr, req_port))
# ask for the sub port
req.send_string("SUB_PORT")
sub_port = req.recv_string()

# open a sub port to listen to pupil
sub = context.socket(zmq.SUB)
sub.connect("tcp://{}:{}".format(addr, sub_port))
sub.setsockopt_string(zmq.SUBSCRIBE, f"surfaces.{surface_name}")

smooth_x, smooth_y = 0.5, 0.5

# screen size
x_dim, y_dim = m.screen_size()
print("x_dim: {}, y_dim: {}".format(x_dim, y_dim))


start
x_dim: 1920, y_dim: 1080


In [ ]:
# Start The BCI

prev_time=time.time()
sfreq=128
info={'sfreq':128}
n_epochs=4000

for ii in range(n_epochs):
#     time.sleep(0.01)

    
    if time.time()-prev_time>2:
        prev_time=time.time()
            
        epoch = client.get_data_as_epoch(n_samples=sfreq,picks=[3,4,5,6,7]);
        data = epoch.get_data();
    #     print(data.shape, np.mean(data))
        chnls1 = [1]
        data = data[:,chnls1,:]
        data1 = wiener(data, (1,1, 40));
        data2 = data1.reshape((1,128))
        data3 = torch.from_numpy(data2).float()
        output = net(data3)
    
#         print(output)
#         if output[0,0]>output[0,1]:
#             pyautogui.click(button='right', clicks=1, interval=0.05)
#         else:
#             pyautogui.click(button='left', clicks=1, interval=0.05) 

    
###########################################    

    sub
    topic, msg = sub.recv_multipart()

    gaze_position = loads(msg, raw=False)
    
    if gaze_position["name"] == surface_name:
        gaze_on_screen = gaze_position["gaze_on_surfaces"]
#         print(gaze_on_screen)

        if len(gaze_on_screen) > 0:

            # there may be multiple gaze positions per frame, so you could average them
            # raw_x = sum([i['norm_pos'][0] for i in gaze_on_screen])/len(gaze_on_screen)
            # raw_y = sum([i['norm_pos'][1] for i in gaze_on_screen])/len(gaze_on_screen)

            # or just use the most recent gaze position on the surface
            raw_x, raw_y = gaze_on_screen[-1]["norm_pos"]

            # smoothing out the gaze so the mouse has smoother movement
            smooth_x += 0.35 * (raw_x - smooth_x)
            smooth_y += 0.35 * (raw_y - smooth_y)
            x = smooth_x
            y = smooth_y

            y = 1 - y  # inverting y so it shows up correctly on screen
            x *= int(x_dim)
            y *= int(y_dim)
            # PyMouse or MacOS bugfix - can not go to extreme corners because of hot corners?
            x = min(x_dim - 10, max(10, x))
            y = min(y_dim - 10, max(10, y))

            # print "%s,%s\n" %(x,y)
#             pyautogui.moveTo(int(x), int(y), duration = .01)
            m.move(int(x), int(y))


Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
1 matching events found
No baselin